Ładowanie zbiorów danych

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pajkel","key":"4043d8516fb9dd6d0a9a3ebd9be36573"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!pip install kaggle


In [4]:
!kaggle datasets download -d meruvulikith/190k-spam-ham-email-dataset-for-classification

Dataset URL: https://www.kaggle.com/datasets/meruvulikith/190k-spam-ham-email-dataset-for-classification
License(s): MIT
 96% 103M/107M [00:01<00:00, 81.4MB/s] 
100% 107M/107M [00:01<00:00, 69.7MB/s]


In [5]:
!unzip 190k-spam-ham-email-dataset-for-classification.zip

Archive:  190k-spam-ham-email-dataset-for-classification.zip
  inflating: spam_Emails_data.csv    


In [6]:
!kaggle datasets download -d purusinghvi/email-spam-classification-dataset

Dataset URL: https://www.kaggle.com/datasets/purusinghvi/email-spam-classification-dataset
License(s): MIT
 77% 33.0M/43.0M [00:00<00:00, 83.5MB/s]
100% 43.0M/43.0M [00:00<00:00, 83.1MB/s]


In [7]:
!unzip email-spam-classification-dataset.zip

Archive:  email-spam-classification-dataset.zip
  inflating: combined_data.csv       


Tworzenie nowego zbioru danych z dwóch pozostałych

In [8]:
import pandas as pd
first_data_set =pd.read_csv('combined_data.csv')
print(first_data_set.head())

   label                                               text
0      1  ounce feather bowl hummingbird opec moment ala...
1      1  wulvob get your medircations online qnb ikud v...
2      0   computer connection from cnn com wednesday es...
3      1  university degree obtain a prosperous future m...
4      0  thanks for all your answers guys i know i shou...


In [9]:
import pandas as pd
second_data_set =pd.read_csv('spam_Emails_data.csv')
print(second_data_set.head())

  label                                               text
0  Spam  viiiiiiagraaaa\nonly for the ones that want to...
1   Ham  got ice thought look az original message ice o...
2  Spam  yo ur wom an ne eds an escapenumber in ch ma n...
3  Spam  start increasing your odds of success & live s...
4   Ham  author jra date escapenumber escapenumber esca...


Ujednolicenie danych w kolumnie label

In [10]:
second_data_set['label'] = second_data_set['label'].replace({'Spam': 1, 'Ham': 0})

In [11]:
print(second_data_set.head())

   label                                               text
0      1  viiiiiiagraaaa\nonly for the ones that want to...
1      0  got ice thought look az original message ice o...
2      1  yo ur wom an ne eds an escapenumber in ch ma n...
3      1  start increasing your odds of success & live s...
4      0  author jra date escapenumber escapenumber esca...


In [12]:
combined_data_set = pd.concat([first_data_set, second_data_set], ignore_index=True)
print(combined_data_set.head())

   label                                               text
0      1  ounce feather bowl hummingbird opec moment ala...
1      1  wulvob get your medircations online qnb ikud v...
2      0   computer connection from cnn com wednesday es...
3      1  university degree obtain a prosperous future m...
4      0  thanks for all your answers guys i know i shou...


In [13]:
print(combined_data_set.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277300 entries, 0 to 277299
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   277300 non-null  int64 
 1   text    277298 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.2+ MB
None


In [14]:
print(combined_data_set.describe())

               label
count  277300.000000
mean        0.489008
std         0.499880
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000


Sprawdzenie czy w zbiorze są puste wartości oraz ich ewentualne usunięcie

In [15]:
nan_counts = combined_data_set.isna().sum()
print(nan_counts)
print(combined_data_set.info())

label    0
text     2
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277300 entries, 0 to 277299
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   277300 non-null  int64 
 1   text    277298 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.2+ MB
None


In [16]:
combined_data_set.dropna(inplace=True)

nan_counts_after_cleaning = combined_data_set.isna().sum()
print(nan_counts_after_cleaning)
print(combined_data_set.info())
combined_data_set.nunique()

label    0
text     0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 277298 entries, 0 to 277299
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   277298 non-null  int64 
 1   text    277298 non-null  object
dtypes: int64(1), object(1)
memory usage: 6.3+ MB
None


label         2
text     193848
dtype: int64

Usunięcie duplikatów z kolumny text

In [17]:
combined_data_set.drop_duplicates(subset=['text'], inplace=True)
combined_data_set.nunique()

label         2
text     193848
dtype: int64

In [18]:
print(combined_data_set.info())

<class 'pandas.core.frame.DataFrame'>
Index: 193848 entries, 0 to 277299
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   label   193848 non-null  int64 
 1   text    193848 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.4+ MB
None


Usuwanie szumów - adresy url jeśli wystąpią

In [19]:
import re

def remove_urls(text):
    url_pattern = r'https?://\S+|www\.\S+'
    return re.sub(url_pattern, '', text)

combined_data_set['filtered_text'] = combined_data_set['text'].apply(remove_urls)
print(combined_data_set.info())

<class 'pandas.core.frame.DataFrame'>
Index: 193848 entries, 0 to 277299
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   label          193848 non-null  int64 
 1   text           193848 non-null  object
 2   filtered_text  193848 non-null  object
dtypes: int64(1), object(2)
memory usage: 5.9+ MB
None


Normalizacja - zamiana wszystkich słów na małe litery, usuwanie znaków specjalnych i interpunkcyjnych

In [20]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text


combined_data_set['normalized_text'] = combined_data_set['filtered_text'].apply(normalize_text)
print(combined_data_set.info())
print(combined_data_set.head())


<class 'pandas.core.frame.DataFrame'>
Index: 193848 entries, 0 to 277299
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   label            193848 non-null  int64 
 1   text             193848 non-null  object
 2   filtered_text    193848 non-null  object
 3   normalized_text  193848 non-null  object
dtypes: int64(1), object(3)
memory usage: 7.4+ MB
None
   label                                               text  \
0      1  ounce feather bowl hummingbird opec moment ala...   
1      1  wulvob get your medircations online qnb ikud v...   
2      0   computer connection from cnn com wednesday es...   
3      1  university degree obtain a prosperous future m...   
4      0  thanks for all your answers guys i know i shou...   

                                       filtered_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer conn

Tokenizacja danych - podzielenie tekstu na mniejsze jednostki – słowa

In [21]:
def tokenize_text(text):
    tokens = text.split()
    return tokens

combined_data_set['tokenized_text'] = combined_data_set['normalized_text'].apply(tokenize_text)

print(combined_data_set.info())
print(combined_data_set.head())

<class 'pandas.core.frame.DataFrame'>
Index: 193848 entries, 0 to 277299
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   label            193848 non-null  int64 
 1   text             193848 non-null  object
 2   filtered_text    193848 non-null  object
 3   normalized_text  193848 non-null  object
 4   tokenized_text   193848 non-null  object
dtypes: int64(1), object(4)
memory usage: 8.9+ MB
None
   label                                               text  \
0      1  ounce feather bowl hummingbird opec moment ala...   
1      1  wulvob get your medircations online qnb ikud v...   
2      0   computer connection from cnn com wednesday es...   
3      1  university degree obtain a prosperous future m...   
4      0  thanks for all your answers guys i know i shou...   

                                       filtered_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircat

Filtracja słów - usuwanie słów nieistotnych

In [22]:
!pip install nltk


In [23]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [24]:
stop_words = set(stopwords.words('english'))

In [25]:
def remove_stopwords(tokens):
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

In [26]:
combined_data_set['filtered_tokens'] = combined_data_set['tokenized_text'].apply(remove_stopwords)


In [27]:
print(combined_data_set.head())

   label                                               text  \
0      1  ounce feather bowl hummingbird opec moment ala...   
1      1  wulvob get your medircations online qnb ikud v...   
2      0   computer connection from cnn com wednesday es...   
3      1  university degree obtain a prosperous future m...   
4      0  thanks for all your answers guys i know i shou...   

                                       filtered_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer connection from cnn com wednesday es...   
3  university degree obtain a prosperous future m...   
4  thanks for all your answers guys i know i shou...   

                                     normalized_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer connection from cnn com wednesday es...   
3  university degree obtain a prosperous future m...   
4  t

Lemantyzacja – stemming to redukcja słów do ich rdzeni (formy bazowej), z uwzględnieniem kontekstu słowa

In [28]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 29.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [29]:
import spacy

nlp = spacy.load('en_core_web_sm')
nlp.max_length = 15000000

In [30]:
def lemmatize_tokens(tokens):
    text = ' '.join(tokens)
    lemmatized = []
    # Przetwarzaj tekst partiami
    for i in range(0, len(text), 1000000):  # Długość partii: 1 000 000 znaków
        doc = nlp(text[i:i+1000000])
        lemmatized.extend([token.lemma_ for token in doc])
    return lemmatized

In [31]:
combined_data_set['lemmatized_text'] = combined_data_set['filtered_tokens'].apply(lemmatize_tokens)

In [32]:
print(combined_data_set.head())

   label                                               text  \
0      1  ounce feather bowl hummingbird opec moment ala...   
1      1  wulvob get your medircations online qnb ikud v...   
2      0   computer connection from cnn com wednesday es...   
3      1  university degree obtain a prosperous future m...   
4      0  thanks for all your answers guys i know i shou...   

                                       filtered_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer connection from cnn com wednesday es...   
3  university degree obtain a prosperous future m...   
4  thanks for all your answers guys i know i shou...   

                                     normalized_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer connection from cnn com wednesday es...   
3  university degree obtain a prosperous future m...   
4  t

Zapisania data setu do pliku

In [34]:
combined_data_set.to_csv('ready_data_set.csv', index=False)

In [35]:
from google.colab import files

files.download('ready_data_set.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>